# Семинар 12. Прунинг, квантование, дистилляция 📱

Хотим запускать нейросети на мобильных устройствах:
- Это дешевле и проще, чем серверное решение
- Надёжнее с точки зрения утечки данных

Какие у нас есть подходы?
- Изменяем архитектуру сети:
  - Просто берём новую архитектуру попроще; ← не поддаётся автоматизации, поэтому не будем обсуждать
  - Дистилляция;
  - Прунинг;
- Не изменяем архитектуру сети:
  - Квантование.

## Словарик

**Дистилляция** - это метод обучения, в процессе которого мы используем большую сеть-учителя, натренированную на данной задаче, чтобы она помогла нам лучше обучить более лёгкую сеть-ученика.

**Прунинг** - это метод, в результате которого мы получаем исходную сеть, из которой удалили наименее важные каналы. Если не слишком сильно "порезать" исходную сеть, то можно её дотренировать, получив значительное ускорение при небольшой просадке в метрике (особенно если сеть для прунинга была изначально "жирной")

**Квантование** - это метод, позволяющий сократить вычислительную сложность и занимаемую память на инференсе нейросети, когда веса и активации представлены в другом типе данных, имеющем меньшую точность (обычно заменяют float32 → int8).

## Дистилляция

![](images/1.png)

**Идея**: используем большую и сложную модель (или ансамбль моделей) для того, чтобы обучить маленькую модель. Маленькая модель будет учиться не на лейблах датасета (или не только на них), а на предсказаниях модели-учителя

**Почему это работает**: обычная разметка - например, 0 или 1 на задаче классификации - не говорит ничего о том, насколько объекты некоторых классов похожи друг на друга. При обучении на таргетах датасета надо будет с нуля учиться понимать, например, что класс "хаски" больше похож на "волка" и меньше похож на "чихуахуа". Маленькой модели должно быть проще научиться "повторять" за моделью-учителем предсказания, чем вытаскивать из данных все эти сложные закономерности.

**Как делать**:
- Добавлять в лосс слагаемое, в котором сравниваются выходы учителя и ученика;
- Можно сравнивать промежуточные выходы, а не только предсказания сети;

![](images/2.png)

*Пример: софтмакс с температурой*

**Пример из жизни**: использовать дистилляцию для обучения модели на эмбеддинги лиц (ResNet100 → MobileFaceNet)

## Прунинг

**Идея**: удалим наименее важные каналы из сети, тогда число операций уменьшится

**Виды прунинга:**

- **Неструктурированный** - зануление ненужных весовых коэффициентов.     
 Если просто заменить значения нулями, то как будто бы ничего не поменяется (всё равно будут делаться арифметические операции, только уже с нулём), надо "выбросить" эти веса → нужно использовать библиотеку для sparse вычислений, хорошее ускорение при очень высоком уровне прореживания
- **Структурированный** - удаление ненужных фильтров или нейронов

**Критерии прунинга:**

- $L_1/L_2$-норма - чем меньше вес, тем меньший вклад он вносит в результат вычислений и тем более вероятно, что его можно выбросить
- Коэффициент масштабирования $\gamma$ у BatchNorm - чем меньше это значение, тем меньше результат на выходе конкретного канала в последующей карте признаков и тем более вероятно, что его можно выбросить
- Группа критериев на основе разложения Тейлора:  
  

Ряд Тейлора:

![](images/4.png)

**Почему это работает**:

- Предположим, у нас есть сеть, обученная на нашей задаче. Рассмотрим вклад какого-то весового коэффициента $W$. Запишем формулу для фунции потерь как функции этого веса: $𝑳(W = W_0)$
- При помощи ряда Тейлора выразим $L(W = 0)$ - значение функции потерь при занулении этого веса
- Если функция потерь не особо поменялась при занулении веса, то как будто бы он не так важен, можно выбросить

Алгоритм прунинга:

1. Берете обученную нейронку;
2. Прогоняете батч на forward и backward;
3. Повторяете, пока не накопите некоторое количество градиентов;
4. Умножаете вес на градиент, который ему соответствует, сортируете и выбрасываете наименьший по критерию.

![](images/5.png)

**Пример из жизни**: запрунили ResNet50 на ImageNet-100k

baseline acc: 76,13%, MMACs: 4144854528
pruned acc: 75,5%, MMACs: 2057615664

speedup: x2.014, accuracy drop 0,63%

## Квантование

**Идея**: используем менее точное представление весов и активаций сети для того, чтобы сэкономить память и уменьшить сложность операций на инференсе

1. Масштабируем тензор $X$ типа float32 так, чтобы у него все значения лежали в $[-1, 1]$.
2. Домножаем на необходимую разрядность
3. Округляем до целого
4. Делаем $clip$

Формулы:

$$S = \frac{I_b}{F_b}, F_b = max(|X|), I_b = 2^n-1,$$ где $n$ - число битов, $X$ - тензор для квантования

$$X_{int} = [S X],$$

где $[\cdot]$ - это операция округления до целого. Тогда квантованный $X$ равен

$$X_q = clip(X_{int}, -I_b, I_b)$$

Как получить из квантованного $X_q$ "обычный" $X$: $$X \approx S \cdot X_q$$

Почему int8 лучше, чем float32?

- Размер модели становится меньше
- Уменьшается потребление памяти, поэтому можно поставить batch_size побольше
- Работают быстрее

**Почему это работает**:

Посчитаем $Y = X\times W$:

$$Y = \frac{S_x X}{S_x} \times \frac{S_w W}{S_w} ≈ \frac{1}{S_x S_w} [S_x X] \times [S_w W] ≈ \frac{X_q\times W_q}{S_x S_w}$$

$X_q, W_q$ - квантованные X и W (в int8)

Затем у нас идет активация ($ReLU$):

$$ReLU(S_xX) = S_xReLU(X)$$

Дальше умножаем $Y\times U$:

$$Y\times U = \frac{S_y Y}{S_y}\times\frac{S_u U}{S_u} \approx \frac{S_y}{S_y} \frac{X_q\times W_q}{S_x S_w}\times \frac{S_u U}{S_u} ≈ \frac{1}{S_y S_u} [\frac{S_y (X_q \times W_q)}{S_x S_w}] \times [S_u U] = \frac{1}{S_y S_u} [(\frac{S_y}{S_x S_w}) (X_q\times W_q)] \times U_q$$ → Мы пришли к перемножению двух квантованных переменных с новыми коэффициентиками $S_y^\prime$ и $S_u$

Получается, что все свёртки и перемножения мы можем производить с квантованными переменными, а при помощи коэффициентов $S$ перевести наш ответ (предсказание сети) назад в тип float32.

*Важное замечание*: наши веса во float32 могут иметь значительные выбросы в распределении, из-за которых предложенная схема квантования с делением на максимальный элемент тензора не будет оптимальной. Лучшим решением будет перенести порог квантования так, чтобы большие выбросы были за пределами порога (см. картинку ниже). Тогда меньший диапазон значений перейдёт в int8, и дискретные "столбики" будут более частыми в той области, где данных много.


![](images/7.png)

**Как делать**:

- Использовать OpenVINO (CPU), TensorRT (GPU) или OnnxRuntime (CPU/GPU)
- Нужен калибровочный датасет. Прогнав этот калибровочный датасет через сеть, OpenVINO/TensorRT определит статистики (как распределены веса на разных слоях), расставит пороги квантования каким-то своим способом и заквантует сеть

**Пример из жизни**

![](images/8.png)